___Import Necessary Packages :___

In [1]:
import numpy as np
import keras
import tensorflow as tf
import cv2
import imutils
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Activation,MaxPooling2D,Flatten,Dense,Dropout
from keras.models import Model,load_model
from keras.callbacks import TensorBoard,ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import Accuracy
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

___Data Augmentation :___

In [2]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib as mt
train_datagen=ImageDataGenerator(rescale=1.0/255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1.0/255)

___Read the Train directory of images :___

In [3]:
Train_Data=train_datagen.flow_from_directory('/content/drive/MyDrive/Train_Data',target_size=(150,150),batch_size=10,class_mode='binary')

Found 500 images belonging to 2 classes.


___Read the Test directory of images :___

In [4]:
Test_Data=test_datagen.flow_from_directory('/content/drive/MyDrive/Test_Data',target_size=(150,150),batch_size=10,class_mode='binary')

Found 100 images belonging to 2 classes.


___CNN Model design :___

In [5]:
Model=Sequential([Conv2D(100,(3,3),activation='relu',input_shape=(150,150,3)),MaxPooling2D(2,2),
                  Conv2D(100,(3,3),activation='relu'),MaxPooling2D(2,2),
                  Flatten(),
                  Dropout(0.5),
                  Dense(50,activation='relu'),
                  Dense(2,activation='softmax')])

___Define the Compiler of CNN :___

In [6]:
Model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [7]:
checkpoint=ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

___Train the model with the data :___

In [8]:
histroy=Model.fit_generator(Train_Data,steps_per_epoch=5,epochs=10,validation_data=Test_Data,validation_steps=4)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
5/5 [==============================] - 20s 1s/step - loss: 1.0454 - acc: 0.4531 - val_loss: 0.6887 - val_acc: 0.4750
Epoch 2/10
5/5 [==============================] - 5s 1s/step - loss: 0.6800 - acc: 0.5450 - val_loss: 0.6882 - val_acc: 0.5500
Epoch 3/10
5/5 [==============================] - 5s 1s/step - loss: 0.6794 - acc: 0.6514 - val_loss: 0.6793 - val_acc: 0.5500
Epoch 4/10
5/5 [==============================] - 5s 1s/step - loss: 0.7144 - acc: 0.4028 - val_loss: 0.6888 - val_acc: 0.4500
Epoch 5/10
5/5 [==============================] - 5s 1s/step - loss: 0.6917 - acc: 0.4264 - val_loss: 0.6689 - val_acc: 0.4250
Epoch 6/10
5/5 [==============================] - 5s 1s/step - loss: 0.6825 - acc: 0.5061 - val_loss: 0.6469 - val_acc: 0.5250
Epoch 7/10
5/5 [==============================] - 5s 1s/step - loss: 0.6749 - acc: 0.4767 - val_loss: 0.6824 - val_acc: 0.4750
Epoch 8/10
5/5 [==============================] - 5s 1s/step - loss: 0.6738 - acc: 0.6792 - val_loss: 0.5456 -

In [9]:
Model.save('mask_project.h5')

___Implement Face Mask Detection with opencv-Haar cascade classifier :___

In [ ]:
model=tf.keras.models.load_model('mask_project.h5')
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size=4
cap=cv2.VideoCapture(0)
haarcascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    (rval,im)=cap.read()
    im=cv2.flip(im,1,1)
    rerect_size=cv2.resize(im,(im.shape[1]//rect_size,im.shape[0]//rect_size))
    faces=haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x,y,w,h)=[v*rect_size for v in f]
        face_img=im[y:y+h,x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped=np.vstack([reshaped])
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im,results[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',im)
    key=cv2.waitKey(10)
    if key==27:
        break
cap.release()
cv2.destroyAllWindows()